In [2]:
import pandas as pd
import json
import os
import re

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Increase the width of the display
pd.set_option('display.max_colwidth', None)  # Don't truncate text in columns

DATA_SRC = "../sources"  
schema_file_path = "mediated_schema.json"  
with open(schema_file_path, "r", encoding="utf-8") as f:
    schema_data = json.load(f)

companies_keys = list(schema_data["schema"].keys())
companies_df = pd.DataFrame(columns=companies_keys)

employees_keys = list(schema_data['employees'].keys())
emp_df = pd.DataFrame(columns=employees_keys)

In [3]:
display(companies_df)

,company_id,company_name,trade_name,industry,sector,categories,company_status,company_type,headquarters,address,postal_code,city,country,state,continent,foundation_year,registration_date,website,rank,market_cap_or_valuation,number_of_employees,ceo,founder,assets,profit_or_net_income,revenue,total_funding_raised,share_price,change_1_day,change_1_year,debts,phone,sic_code,emtak_code,nace_code,facebook,twitter,pinterest,instagram,investors,region,notes_or_description


In [4]:
display(emp_df)

,idAzienda,nameEmployee,code,role,startDate,participation,contribution


In [7]:
schema_mapping_path = "schema_mapping.json"
with open(schema_mapping_path, "r", encoding="utf-8") as f:
    field_mapping = json.load(f)

In [8]:
def normalize_strings(value):
    if isinstance(value, str):
        value = re.sub(r'\s+', ' ', value.strip())
        value = value.replace('\\n', '').strip()
        value = value.lower()
    return value

In [13]:
def map_source_to_schema(source_df, field_mapping):
    source_columns = field_mapping.keys()
    mapped_df = source_df[list(source_columns)].copy()
    mapped_df.rename(columns=field_mapping, inplace=True)
    mediated_schema_columns = list(set(field_mapping.values()))

    for column in mediated_schema_columns:
        if column not in mapped_df.columns:
            mapped_df[column] = None
    return mapped_df

In [17]:
def integrate_data(companies_df, transformed_df):
    for column in transformed_df.columns:
        if column not in companies_df.columns:
            companies_df[column] = None  # Add missing columns with None

    # Concatenate along rows (axis=0), ignoring index to avoid conflicts
    integrated_df = pd.concat([companies_df, transformed_df], ignore_index=True)

    return integrated_df

In [18]:
for source_file in os.listdir(DATA_SRC):
    file_path = os.path.join(DATA_SRC, source_file)
    
    if file_path.endswith('.csv'):
        try:
            source_df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            source_df = pd.read_csv(file_path, encoding='ISO-8859-1')
    elif file_path.endswith('.json'):
        with open(file_path, 'r') as f:
            data = json.load(f)
        if isinstance(data, list):
            source_df = pd.json_normalize(data)
        else:
            source_df = pd.json_normalize([data])
    elif file_path.endswith('.xls'):
        source_df = pd.read_excel(file_path)
    elif file_path.endswith('.jsonl'):
        source_df = pd.read_json(file_path, lines=True)

    datasource = os.path.splitext(source_file)[0]
    if source_file.startswith('wissel-partners') or source_file.startswith('wissel-rappresentanti'):
        temp = map_source_to_schema(source_df, field_mapping['employees_mapping'][datasource])
    else:
        temp = map_source_to_schema(source_df, field_mapping['companies_mapping'][datasource])

    temp = integrate_data(companies_df, temp)

display(temp.head(1))
     


,id,name,address,nation,hhid,industry,sic_code,type,est_of_ownership
0,d60823f20d864692b517a8ad6c1418ed,Enel Spa,"VIALE REGINA MARGHERITA 137, ROMA, 00198, ROMA, ITALY",IT00934061003,H-IT0031426662,"Finance, Insurance, and Real Estate",Holding company (6719),Corporation,1962


,company_id,company_name,trade_name,industry,sector,categories,company_status,company_type,headquarters,address,postal_code,city,country,state,continent,foundation_year,registration_date,website,rank,market_cap_or_valuation,number_of_employees,ceo,founder,assets,profit_or_net_income,revenue,total_funding_raised,share_price,change_1_day,change_1_year,debts,phone,sic_code,emtak_code,nace_code,facebook,twitter,pinterest,instagram,investors,region,notes_or_description
0,H-IT0031426662,Enel Spa,NaN,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,Corporation,NaN,"VIALE REGINA MARGHERITA 137, ROMA, 00198, ROMA, ITALY",NaN,NaN,IT00934061003,NaN,NaN,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Holding company (6719),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
